In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
workspace = '/content/gdrive/MyDrive/ONNXAnalyser/onnx_models'

Mounted at /content/gdrive


In [2]:
!pip install onnx numpy
!pip install onnxruntime
!pip install plotly==4.14.3
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 75.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79

In [ ]:
import onnx
import numpy as np
import onnxruntime as ort

def print_intermediate_results(model_path, input_data):
    model = onnx.load(model_path)
    graph = model.graph

    # collect intermediate output names
    output_names = []
    for i, item in enumerate(graph.node):
      output_names.append(item.output[0])
    # add intermediate outputs to onnx model
    shape_info = onnx.shape_inference.infer_shapes(model)
    value_info_protos = []
    shape_info = onnx.shape_inference.infer_shapes(model)
    for value_info in shape_info.graph.value_info:
      if value_info.name in output_names:
        value_info_protos.append(value_info)
    graph.output.extend(value_info_protos)

    # Perform shape inference on the model
    inferred_model = onnx.shape_inference.infer_shapes(model)

    graph = inferred_model.graph

    # save the modified onnx model
    modified_model_path = workspace + "/modified_model.onnx"
    onnx.save(model, modified_model_path)
    # modified temp model
    model = onnx.load(modified_model_path)
    graph = model.graph
    # Collect intermediate results
    intermediate_results = {}
    graph_output_names = [output.name for output in graph.output]
    for node in graph.node:
        # Check if the node output is an intermediate result
        # Run inference for the node and retrieve the outputs
        session = ort.InferenceSession(modified_model_path)
        input_name = session.get_inputs()[0].name
        output_names = [output for output in graph_output_names]
        outputs = session.run(output_names, {input_name: input_data})

        # Store the intermediate results
        for name, output in zip(output_names, outputs):
            intermediate_results[name] = output
    memory_bandwidth = 0
    index = 0
    # Print intermediate results
    for name, result in intermediate_results.items():
        if 'batchnorm' in name or 'relu' in name:
          continue
        print(f'{index} Intermediate: {name}\tShape: {result.shape}\tSize: {result.size}')
        index += 1
        memory_bandwidth += result.size * 2
    print("memory / bandwidth: {}".format(memory_bandwidth))
    return memory_bandwidth

# Path to the ONNX model file
model_path = workspace + '/test_model.onnx'
# # Load the model weights (example)
# weights_dict = {}
# for name, weight in zip(model.graph.input, model.graph.initializer):
#     weights_dict[name] = np.array(weight)
# Input data as a NumPy array
input_data = np.random.randn(1, 3, 224, 224).astype(np.float32)  # Adjust the shape and data type as per your model's input requirements

# Print intermediate results
print_intermediate_results(model_path, input_data)


0 Intermediate: resnetv15_dense0_fwd	Shape: (1, 1000)	Size: 1000
1 Intermediate: resnetv15_conv0_fwd	Shape: (1, 64, 112, 112)	Size: 802816
2 Intermediate: resnetv15_pool0_fwd	Shape: (1, 64, 56, 56)	Size: 200704
3 Intermediate: resnetv15_stage1_conv0_fwd	Shape: (1, 64, 56, 56)	Size: 200704
4 Intermediate: resnetv15_stage1_conv1_fwd	Shape: (1, 64, 56, 56)	Size: 200704
5 Intermediate: resnetv15_stage1__plus0	Shape: (1, 64, 56, 56)	Size: 200704
6 Intermediate: resnetv15_stage1_activation0	Shape: (1, 64, 56, 56)	Size: 200704
7 Intermediate: resnetv15_stage1_conv2_fwd	Shape: (1, 64, 56, 56)	Size: 200704
8 Intermediate: resnetv15_stage1_conv3_fwd	Shape: (1, 64, 56, 56)	Size: 200704
9 Intermediate: resnetv15_stage1__plus1	Shape: (1, 64, 56, 56)	Size: 200704
10 Intermediate: resnetv15_stage1_activation1	Shape: (1, 64, 56, 56)	Size: 200704
11 Intermediate: resnetv15_stage2_conv2_fwd	Shape: (1, 128, 28, 28)	Size: 100352
12 Intermediate: resnetv15_stage2_conv0_fwd	Shape: (1, 128, 28, 28)	Size: 100

8383440

In [ ]:
import os
onnx_models = os.listdir(workspace)

onnx_models = ['bvlcalexnet-12.onnx','vgg16-12.onnx','vgg19-7.onnx', 'inception-v1-12.onnx', 'inception-v2-9.onnx',
               'googlenet-12.onnx','resnet18-v1-7.onnx', 'resnet34-v1-7.onnx', 'resnet50-v1-7.onnx', 'resnet101-v1-7.onnx', 'resnet152-v1-7.onnx',
               'densenet-9.onnx', 'squeezenet1.0-12.onnx', 'mobilenetv2-7.onnx', 'shufflenet-v2-10.onnx', 'efficientnet-lite4-11.onnx']

# check again
# 'bvlcalexnet-3.onnx',
memory_estimate = {}
count = 0
for onnx_model in onnx_models:
  print(onnx_model)

for onnx_model in onnx_models:
  # Path to the ONNX model file
  if 'onnx' not in onnx_model:
    continue
  print('Model:', onnx_model)
  model_path = workspace + '/' + onnx_model
  # Input data as a NumPy array
  if 'efficientnet' in onnx_model:
    input_data = np.random.randn(1, 224, 224, 3).astype(np.float32)
  else:
    input_data = np.random.randn(1, 3, 224, 224).astype(np.float32)  # Adjust the shape and data type as per your model's input requirements
  # Print intermediate results
  memory_estimate[onnx_model[:-5]] = print_intermediate_results(model_path, input_data)
  # if count == 5:
  #   break;
  # count += 1



bvlcalexnet-12.onnx
vgg16-12.onnx
vgg19-7.onnx
inception-v1-12.onnx
inception-v2-9.onnx
googlenet-12.onnx
resnet18-v1-7.onnx
resnet34-v1-7.onnx
resnet50-v1-7.onnx
resnet101-v1-7.onnx
resnet152-v1-7.onnx
densenet-9.onnx
squeezenet1.0-12.onnx
mobilenetv2-7.onnx
shufflenet-v2-10.onnx
efficientnet-lite4-11.onnx
Model: bvlcalexnet-12.onnx
0 Intermediate: prob_1	Shape: (1, 1000)	Size: 1000
1 Intermediate: conv1_1	Shape: (1, 96, 54, 54)	Size: 279936
2 Intermediate: conv1_2	Shape: (1, 96, 54, 54)	Size: 279936
3 Intermediate: norm1_1	Shape: (1, 96, 54, 54)	Size: 279936
4 Intermediate: pool1_1	Shape: (1, 96, 26, 26)	Size: 64896
5 Intermediate: conv2_1	Shape: (1, 256, 26, 26)	Size: 173056
6 Intermediate: conv2_2	Shape: (1, 256, 26, 26)	Size: 173056
7 Intermediate: norm2_1	Shape: (1, 256, 26, 26)	Size: 173056
8 Intermediate: pool2_1	Shape: (1, 256, 12, 12)	Size: 36864
9 Intermediate: conv3_1	Shape: (1, 384, 12, 12)	Size: 55296
10 Intermediate: conv3_2	Shape: (1, 384, 12, 12)	Size: 55296
11 Interme

In [ ]:
# Don't run if you don't want to overwrite results
import pickle as pkl
file_path = workspace + '/memory_estimates.pkl'  # Replace with the desired file path

# Open the file in binary mode for writing
with open(file_path, 'wb') as file:
    pkl.dump(memory_estimate, file)

In [3]:
import pickle as pkl
file_path = workspace + '/memory_estimates.pkl'  # Replace with the desired file path

# Open the file in binary mode for read
with open(file_path, 'rb') as file:
    memory_estimate = pkl.load(file)

In [5]:
import plotly.graph_objects as go

# Extract keys and values from the dictionary
categories = list(memory_estimate.keys())
counts = list(memory_estimate.values())

# Create a bar trace for the histogram
trace = go.Bar(x=categories, y=counts)

# C reate the layout for the histogram
layout = go.Layout(title= {
                      'text': 'Network Architecture vs Memory B/W',
                      'x': 0.5,
                   },
                   xaxis=dict(title='Network Architecture'),
                   yaxis=dict(title='Memory B/W'))

# Create the figure and add the trace
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()
fig.write_image('bandwidth.jpg')
fig.write_html('image.html')
# Save the figure as an image
# fig.write_image("network_vs_memory_bw.png")  # Change the filename and extension as desired